<a href="https://colab.research.google.com/github/vivek6395/My_work/blob/main/LLM_Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-qdrant
!pip install langchain-huggingface
!pip install langchain-community
!pip install huggingface_hub
!pip install fastembed
!pip install qdrant-client
!pip install sentence-transformer
!pip install sentence-transformers

In [50]:
from qdrant_client import QdrantClient, models
from langchain_qdrant import QdrantVectorStore
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_huggingface import HuggingFaceEmbeddings
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

catalan.txt:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

dutch.txt:   0%|          | 0.00/453 [00:00<?, ?B/s]

azerbaijani.txt:   0%|          | 0.00/967 [00:00<?, ?B/s]

...

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

tajik.txt:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
client_path="qdrant/collection"
collection="mycollection"


In [5]:
client=QdrantClient(path=client_path)


In [86]:
client.delete_collection(collection_name=collection)

True

In [87]:
client.create_collection(
    collection_name=collection,
    vectors_config={
        "dense":models.VectorParams(size=len(embeddings.embed_query("hi")),distance=models.Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(),
    }
)

True

In [23]:
import requests
url="https://www.nothuman.net/images/files/discussion/1/e72f9f1f181a66887baa7270037c582e.pdf"

response=requests.get(url=url)
file_name="science_book.pdf"

with open(file_name,"wb") as file:
  file.write(response.content)

In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_name,)
pages = loader.load()

In [62]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1250,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

In [63]:
docs=text_splitter.split_documents(pages)

In [77]:
def preprocess_text(input_str:str):
  lower_case_str = input_str.lower()
  result_str = []
  for char in lower_case_str:
      if unicodedata.category(char).startswith('P'):
          result_str.append(' ')
      else:
          result_str.append(char)
  return ''.join(result_str).replace('\n'," \n ")

In [78]:
import string
import unicodedata

docs=docs[0:250]
small_docs=[]
from langchain_core.documents import Document

for i in docs:
  new_string=preprocess_text(i.page_content)
  my_doc=Document(

      page_content=new_string,
      metadata={
      }
  )
  small_docs.append(my_doc)

In [79]:
small_docs[0:10]

[Document(metadata={}, page_content='a game of thrones \n book one of a song of ice and fire \n by george r r  martin \n contents \n maps \n the north \n the south \n ●prologue \n ●chapter 1 \n ●chapter 2 \n ●chapter 3 \n ●chapter 4 \n ●chapter 5 \n ●chapter 6 \n ●chapter 7 \n ●chapter 8 \n ●chapter 9 \n ●chapter 10 \n ●chapter 11 \n ●chapter 12'),
 Document(metadata={}, page_content='●chapter 13 \n ●chapter 14 \n ●chapter 15 \n ●chapter 16 \n ●chapter 17 \n ●chapter 18 \n ●chapter 19 \n ●chapter 20 \n ●chapter 21 \n ●chapter 22 \n ●chapter 23 \n ●chapter 24 \n ●chapter 25 \n ●chapter 26 \n ●chapter 27 \n ●chapter 28 \n ●chapter 29 \n ●chapter 30 \n ●chapter 31 \n ●chapter 32 \n ●chapter 33 \n ●chapter 34 \n ●chapter 35 \n ●chapter 36 \n ●chapter 37 \n ●chapter 38 \n ●chapter 39 \n ●chapter 40 \n ●chapter 41 \n ●chapter 42 \n ●chapter 43 \n ●chapter 44 \n ●chapter 45 \n ●chapter 46 \n ●chapter 47 \n ●chapter 48 \n ●chapter 49 \n ●chapter 50●chapter 51 \n ●chapter 52 \n ●chapter 53 \n ●

In [88]:
vector_store_sparse=QdrantVectorStore(
    client=client,
    collection_name=collection,
    retrieval_mode=RetrievalMode.SPARSE,
    distance=models.Distance.DOT,
    sparse_embedding=sparse_embeddings,
    sparse_vector_name="sparse",
)

In [89]:
vector_store_dense=QdrantVectorStore(
    client=client,
    collection_name=collection,
    embedding=embeddings,
    vector_name='dense',
)

In [82]:
base=10000
sparse_ids=[i for i in range(len(docs))]
dense_ids=[i+base for i in range(len(docs))]

In [90]:
vector_store_sparse.add_documents(small_docs, ids=sparse_ids)
vector_store_dense.add_documents(docs,ids=dense_ids)
print()

In [92]:
query="Ser Waymar looked him over with open disapproval. “I am not going back to Castle"
query=preprocess_text(query)
print(query)
vector_store_sparse.similarity_search_with_score(query)

ser waymar looked him over with open disapproval   i am not going back to castle


[(Document(metadata={'_id': 17, '_collection_name': 'mycollection'}, page_content=' on your feet  will   ser waymar commanded   there s no one here  i won t have you  \n hiding under a bush   \n reluctantly  will obeyed ser waymar looked him over with open disapproval   i am not going back to castle  \n black a failure on my first ranging  we will find these men   he glanced around   up the tree  be quick about it  look for a fire   \n will turned away  wordless  there was no use to argue  the wind was moving  it cut  \n right through him  he went to the tree  a vaulting grey green sentinel  and began to climb  soon his hands were sticky with sap  and he was lost among the needles  fear filled his gut like a meal he could not digest  he whispered a prayer to the nameless gods of the wood  and slipped his dirk free of its sheath  he put it between his teeth to keep both hands free for climbing  the taste of cold iron in his mouth gave him comfort  \n down below  the lordling called out 